In [5]:
#import and construct hepler function
from util_main import load_dance_dance_data_set
from util_main import test_model_nn
from util_main import load_model as load_model_keras
from util_main import one_hot_encode_labels

import torch.nn as nn
import torch.nn.functional as F


#load data set
[x_data, y_data] = load_dance_data_set()

input_size = x_data.shape[1]
output_size = one_hot_encode_labels(y_data).shape[1]

#3 linear layer with ReLu activation for hidden and softmax activation for output
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.hidden1 = nn.Linear(input_size, 256)
        self.hidden2 = nn.Linear(256, 256)
        self.hidden3 = nn.Linear(256, 128)
        self.output = nn.Linear(128, output_size)
        
    def forward(self, x):
        x = self.hidden1(x)
        x = self.hidden2(F.relu(x))
        x = self.hidden3(F.relu(x))
        x = self.output(F.relu(x))
        return F.softmax(x)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

def prepare_data_loaders():

    dataset = load_dance_data_set()

    train_sampler = torch.utils.data.RandomSampler(dataset)

    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=train_batch_size,
        sampler=train_sampler)

    return data_loader
    
def evaluate(model, criterion, data_loader, neval_batches):
    model.eval()
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    cnt = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data)
            loss = criterion(output, target)
            cnt += 1
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            print('.', end = '')
            top1.update(acc1[0], data.size(0))
            top5.update(acc5[0], data.size(0))
            if cnt >= neval_batches:
                 return top1, top5

    return top1, top5


def accuracy(output, target, topk=(1,)):
        """Computes the accuracy over the k top predictions for the specified values of k"""
        with torch.no_grad():
            maxk = max(topk)
            batch_size = target.size(0)

            _, pred = output.topk(maxk, 1, True, True)
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))

            res = []
            for k in topk:
                correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
                res.append(correct_k.mul_(100.0 / batch_size))
            return res


def load_model(model_file):
    model = Model()
    state_dict = torch.load(model_file)
    model.load_state_dict(state_dict)
    model.to('cpu')
    return model

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

ImportError: cannot import name 'load_dance_dance_set' from 'util_main' (C:\Users\longd\Notebooks\CG4002\MachineLearning\main_code\util_main.py)

In [ ]:
#Model conversion from Keras to Pytorch
! mmconvert -sf Keras -iw neural_network.h5 -in neural_network.json -df pytorch -om converted_model.pth 

model_file = "./model.pth
model_file_keras = "neural_network"
#load converted model here
model = load_model(model_file)
model_keras = load_model_nn(model_file_keras)


#test converted model with orginal model 
y_keras = model_keras.predict(x_data)
y = model(x_data)


In [2]:
criterion = nn.CrossEntropyLoss()
data_loader = prepare_data_loader()

# Specify quantization configuration
# Start with simple min/max range estimation and per-tensor quantization of weights
model.qconfig = torch.quantization.default_qconfig
print(model.qconfig)
torch.quantization.prepare(model, inplace=True)

# Calibration
print('Post Training Quantization Prepare: Inserting Observers')
evaluate(model, criterion, data_loader, neval_batches=num_calibration_batches)
print('Post Training Quantization: Calibration done')

# Convert to quantized model
torch.quantization.convert(myModel, inplace=True)
print('Post Training Quantization: Convert done')

print("Size of model after quantization")
print_size_of_model(myModel)

y_quantized = model(x_data)
errors = np.substract(y, y_quantized)
reduce_accuracy = abs(sum(errors))/errors.shape[1]*100

SyntaxError: invalid syntax (<ipython-input-2-b9a0d37fd484>, line 8)